In [2]:
!pip install pandas
!pip install optuna
!pip install optuna-dashboard
!pip install scikit
!pip install imbalanced-learn
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
    minmax_scale,
)
from sklearn.metrics import recall_score, accuracy_score,f1_score, precision_score, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
import warnings
import optuna
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score

ERROR: Could not find a version that satisfies the requirement scikit (from versions: none)
ERROR: No matching distribution found for scikit


d:\ADL\ADL\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using", device)
def init_weights(model): #tested already
    if isinstance(model, nn.Linear):  # Apply only to linear layers
        nn.init.xavier_uniform_(model.weight)
        if model.bias is not None:
            nn.init.zeros_(model.bias)
            
def fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=64, device=device):
    train_dataset = TensorDataset(
        torch.tensor(train_x.values,dtype=torch.float32).to(device), 
        torch.tensor(train_y.values,dtype=torch.float32).to(device))
    val_dataset = TensorDataset(
        torch.tensor(test_x.values,dtype=torch.float32).to(device), 
        torch.tensor(test_y.values,dtype=torch.float32).to(device))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False, drop_last=True)
    return train_loader, val_loader 

def get_feature_count(loader):
    """returns the number of features in the dataset"""
    return next(iter(loader))[0].shape[1]

from Criterion_Models import *
def criterion_mapping(criterion_choice:str, pos_weight:float=None, alpha:float=None, gamma:float=None):
    """
    Feel free to add any custom loss functions here.
    returns function for criterion
    """
    if criterion_choice == "FocalLoss":
        return FocalLoss(alpha =alpha, gamma=gamma)
    elif criterion_choice == "DiceLoss":
        return DiceLoss()
    elif criterion_choice == "BCEWithLogitsLoss":
        return nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight])) if pos_weight else nn.BCEWithLogitsLoss()
    return nn.BCEWithLogitsLoss() 

Using cpu


In [4]:
random_state = 42
raw_dataset = pd.read_csv("./data/processed_data.csv") #data has X and Y
X = raw_dataset.drop(columns=["DR"])
Y = pd.DataFrame(raw_dataset["DR"])

In [12]:
#drop cols here
# numeric_columns = ['Age', 'UAlb', 'Ucr', 'UACR', 'TC', 'TG', 'TCTG', 'LDLC', 'HDLC', 'Scr', 'BUN', 'FPG', 'HbA1c', 'Height', 'Weight', 'BMI', 'Duration']
# binary_columns = ['Gender', 'DR', 'Community_baihe', 'Community_chonggu', 'Community_huaxin', 'Community_jinze', 'Community_liantang', 'Community_xianghuaqiao', 'Community_xujin', 'Community_yingpu', 'Community_zhaoxian', 'Community_zhujiajiao']

In [5]:
X_FOR_FOLDS, X_FINAL_TEST, Y_FOR_FOLDS, Y_FINAL_TEST = train_test_split(X, Y, test_size=0.1, random_state=random_state, stratify=Y)

In [6]:
def augment_data_in_place(X, X_test, Y=None, normalisation_method=MinMaxScaler(), noise=None):
    all_numerical_columns = ['Age', 'UAlb', 'Ucr', 'UACR', 'TC', 'TG', 'TCTG', 'LDLC', 'HDLC', 'Scr', 'BUN', 'FPG', 'HbA1c', 'Height', 'Weight', 'BMI', 'Duration']
    binary_columns = ['Gender', 'DR', 'Community_baihe', 'Community_chonggu', 'Community_huaxin', 'Community_jinze', 'Community_liantang', 'Community_xianghuaqiao', 'Community_xujin', 'Community_yingpu', 'Community_zhaoxian', 'Community_zhujiajiao']
    
    existing_columns = [col for col in all_numerical_columns if col in X.columns and col in X_test.columns]

    if not existing_columns:
        print("No matching columns found for augmentation. Normalised data only.")
        X = normalisation_method.fit_transform(X)
        X_test = normalisation_method.transform(X_test)
        return X, X_test

    X_copy = X.copy()
    X_test_copy = X_test.copy()
    
    # Log-transform
    X_copy.loc[:, existing_columns] = X_copy.loc[:, existing_columns].apply(np.log1p)
    X_test_copy.loc[:, existing_columns] = X_test_copy.loc[:, existing_columns].apply(np.log1p)

    # Add noise ONLY to negatives (class 0) if Y is provided and noise is set
    if noise and noise > 0:
        if Y is None:
            raise ValueError("Y must be provided if noise is being added selectively.")
        # Identify negative class indices (class 0)
        negative_indices = Y[Y.iloc[:, 0] == 0].index
        noise_matrix = np.random.normal(0, noise, X_copy.loc[negative_indices, existing_columns].shape)
        X_copy.loc[negative_indices, existing_columns] += noise_matrix

    # Scale
    scaler = normalisation_method
    X_copy.loc[:, existing_columns] = scaler.fit_transform(X_copy.loc[:, existing_columns])
    X_test_copy.loc[:, existing_columns] = scaler.transform(X_test_copy.loc[:, existing_columns])

    return X_copy, X_test_copy


def iso_forest(X_train, Y_train, contamination=None, random_state=42):
    # print("Original\n", X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
    X_train_cleaned, Y_train_cleaned = X_train.copy(), Y_train.copy()
    
    X_train_zeros = X_train[Y_train.iloc[:, 0] == 0]
    X_train_ones = X_train[Y_train.iloc[:, 0] == 1]
    Y_train_zeros = Y_train[Y_train.iloc[:, 0] == 0]
    Y_train_ones = Y_train[Y_train.iloc[:, 0] == 1] 
    # print("Ones and zeros\n", X_train_zeros.shape, Y_train_zeros.shape, X_train_ones.shape, Y_train_ones.shape)
    #only class 0s
    if X_train_zeros.isna().any().any():
        print("got NaN values in the training set")
    
    # Apply Isolation Forest to majority class only
    iso_forest = IsolationForest(contamination=contamination, random_state=random_state)
    try:
        outliers = iso_forest.fit_predict(X_train_zeros)
    except UserWarning as e:
        print("Caught warning during IsolationForest fitting:", e)
        outliers = np.ones(len(X_train_zeros))  # If warning occurs, keep all data
    # Keep only non-outlier majority samples
    X_train_zeros = X_train_zeros[outliers == 1]
    Y_train_zeros = Y_train_zeros[outliers == 1]
    # print("After iso:\n", X_train_zeros.shape, Y_train_zeros.shape, X_train_ones.shape, Y_train_ones.shape)
    
    # Combine the cleaned majority class with the untouched minority class
    X_train_cleaned = pd.concat([X_train_zeros, X_train_ones])
    Y_train_cleaned = pd.concat([Y_train_zeros, Y_train_ones])
    return X_train_cleaned, Y_train_cleaned


In [7]:
def FOLDS_GENERATOR(X, Y, normalisation_method=MinMaxScaler(), n_splits=5, random_state=None, oversampler=None, contamination=0.05, noise = None):
    """
    Generates stratified folds with specified normalization.
    normalisation_method should be an instance of a scaler, e.g.,
    - MinMaxScaler()
    Returns a list of tuples, each containing:
    (X_train_scaled, X_test_scaled, Y_train, Y_test), representing data for each fold
    """
    kF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    kFolds_list = []

    for fold, (train_idx, test_idx) in enumerate(kF.split(X, Y)):
        # Split the data into training and testing sets for this fold
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]
        # print("Original\n", X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
        
        if contamination is not None and contamination > 0: #? using contamination = 0.0 works
            X_train_cleaned, Y_train_cleaned = iso_forest(X_train, Y_train, contamination=contamination, random_state=random_state)
        
        #? data augmentation on leftover data
        X_train_scaled, X_test_scaled = augment_data_in_place(X_train_cleaned, X_test, Y_train_cleaned,normalisation_method=normalisation_method, noise = noise)
        
        # Handle oversampling if needed
        #! use X_train_scaled and Y_train_cleaned for oversampling becasue y_train_cleaned no changes after augmentation
        print("Before oversampling class distribution:")
        print(Y_train_cleaned.value_counts())
        if oversampler:
            X_train_scaled, Y_train_cleaned = oversampler.fit_resample(X_train_scaled, Y_train_cleaned)
        print("\nAfter oversampling class distribution:")
        print(Y_train_cleaned.value_counts())
        # Convert scaled data back to DataFrame with the correct column names
        X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_cleaned.columns)
        X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

        # Handle community columns
        community_cols = [col for col in X_train_scaled.columns if col.startswith('Community')]
        if community_cols:
            X_train_scaled[community_cols] = X_train_scaled[community_cols].apply(
                lambda row: pd.Series(np.eye(len(row))[row.argmax()]), axis=1
            ).set_axis(community_cols, axis=1)
        # print(X_train_scaled[community_cols].describe())

        # Ensure 'Gender' is still binary (0 or 1)
        if 'Gender' in X_train_scaled.columns:
            X_train_scaled['Gender'] = (X_train_scaled['Gender'] > 0.5).astype(int)
            X_test_scaled['Gender'] = (X_test_scaled['Gender'] > 0.5).astype(int)

        # Append the processed fold to the list
        kFolds_list.append((X_train_scaled, X_test_scaled, Y_train_cleaned, Y_test))

        print(f"Fold: {fold+1}, Train: {X_train_scaled.shape}, Test: {X_test_scaled.shape}")

    return kFolds_list

In [8]:
oversampler = ADASYN(sampling_strategy='minority', n_neighbors=5, random_state=42)
contamination = 0.05
normalisation_method = MinMaxScaler()
kFolds = FOLDS_GENERATOR(X_FOR_FOLDS, Y_FOR_FOLDS, 
                         normalisation_method = normalisation_method, 
                         n_splits=5, 
                         oversampler = oversampler, random_state=42, contamination=contamination, noise = None)

ValueError: could not convert string to float: '1,156'

In [9]:
for list in kFolds:
    print(list[0].shape, list[1].shape, list[2].shape, list[3].shape)
    print(list[0].describe())
    a = list[0]
    break

NameError: name 'kFolds' is not defined

In [44]:
oversampler = ADASYN(sampling_strategy='minority', n_neighbors=5, random_state=42)
contamination = 0.05
normalisation_method = MinMaxScaler()
kFolds2 = FOLDS_GENERATOR(X_FOR_FOLDS, Y_FOR_FOLDS, 
                         normalisation_method = normalisation_method, 
                         n_splits=5, 
                         oversampler = oversampler, random_state=42, contamination=contamination, noise = 0.2)

Fold: 1, Train: (7825, 28), Test: (1149, 28)
Fold: 2, Train: (7886, 28), Test: (1149, 28)
Fold: 3, Train: (7822, 28), Test: (1148, 28)
Fold: 4, Train: (7836, 28), Test: (1148, 28)
Fold: 5, Train: (7876, 28), Test: (1148, 28)


In [ ]:
for list in kFolds2:
    print(list[0].shape, list[1].shape, list[2].shape, list[3].shape)
    print(list[0].describe())
    break

(7941, 28) (1149, 28) (7941, 1) (1149, 1)
               Age       Gender         UAlb          Ucr         UACR  \
count  7941.000000  7941.000000  7941.000000  7941.000000  7941.000000   
mean      0.550838     0.539227     0.418078     0.458959     0.368753   
std       0.119223     0.498490     0.175028     0.347252     0.157979   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.472435     0.000000     0.300738     0.153257     0.261809   
50%       0.553392     1.000000     0.396253     0.216133     0.347300   
75%       0.629549     1.000000     0.521173     0.845490     0.457313   
max       1.000000     1.000000     1.000000     1.000000     1.000000   

                TC           TG         TCTG         LDLC         HDLC  ...  \
count  7941.000000  7941.000000  7941.000000  7941.000000  7941.000000  ...   
mean      0.446019     0.265299     0.473273     0.541935     0.476491  ...   
std       0.104849     0.095436     0.148022     0.122